## Get Data 
1. Read the historical AFL results data from file. 
2. Drop extra cols, drop NA; Keep Scores, Outcomes, and Aggregated Odds.
3. Analyze 

In [1]:
import pandas as pd 
import json 
import numpy as np

DATA_FILEPATH = "./data/afl.xlsx"
data = pd.read_excel(DATA_FILEPATH, header=1)
data = data[['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Home Odds', 'Away Odds']]
data['Outcomes'] = np.where(data['Home Score'] > data['Away Score'], 1, 0)
data.dropna(subset=['Date', 'Home Odds', 'Away Odds'], inplace=True)
print(f"Data from {data['Date'].min()} to {data['Date'].max()}")
print(f"Total matches: {len(data)}")

Data from 2009-06-19 00:00:00 to 2025-06-22 00:00:00
Total matches: 3264


In [2]:
def calculate_normmalized_implied_probabilities(actual_odds):
    """
    Calculate implied probabilities from odds.
    
    Args:
        odds (pd.DataFrame): DataFrame containing 'Home Odds' and 'Away Odds'.
        
    Returns:
        pd.DataFrame: DataFrame with implied probabilities for home and away teams.
    """
    actual_probs = 1 / actual_odds
    total_prob = actual_probs.sum(axis=1)
    fair_probs = actual_probs.div(total_prob, axis=0)
    fair_probs.columns = ['Home Probs', 'Away Probs'] 
    fair_odds = 1 / fair_probs
    fair_odds.columns = ['Home Odds', 'Away Odds']
    return fair_odds, fair_probs

def calculate_weighted_implied_probabilities(actual_odds):
    return None

def simulate_favourite_betting_strategy(odds, probs, outcomes, stake=10):
    """
    Simulate a betting strategy where you bet on the favourite team.
    
    Args:
        odds (pd.DataFrame): DataFrame containing 'Home Odds' and 'Away Odds'.
        probs (pd.DataFrame): DataFrame containing 'Home Probs' and 'Away Probs'.
        outcomes (pd.Series): Series with actual outcomes (1 for Home win, 0 for Away).
        stake (int): Amount to bet each time.
        
    Returns:
        np.ndarray: Array of profits for each bet. 
    """
    home_odds = odds["Home Odds"].values
    away_odds = odds["Away Odds"].values

    # Identify the favourite team based on odds
    is_home_favourite = home_odds < away_odds
    favourite_odds = np.where(is_home_favourite, home_odds, away_odds)

    # Determin if favourite team won
    favourite_wins = np.where(is_home_favourite, outcomes, 1 - outcomes)

    # Calculate total winnings
    profits = np.where(favourite_wins, (favourite_odds * stake) - stake, -stake)
    total_wins = np.sum(favourite_wins)

    return profits, total_wins 

def calculate_brier_score(probs, outcomes):
    """
    Calculate the Brier Score for the given probabilities and outcomes.
    """
    # Step 1: Brier Score (2-class: Home vs Away)
    probs = np.vstack([probs['Home Probs'], probs['Away Probs']]).T
    actuals = np.vstack([outcomes, 1 - outcomes]).T
    brier_score = np.mean(np.sum((probs - actuals) ** 2, axis=1))

    return brier_score

In [3]:
# Get fair odds - Normalized odds
fair_odds, fair_probs = calculate_normmalized_implied_probabilities(data[['Home Odds', 'Away Odds']])

# Get Brier Score
brier_score = calculate_brier_score(fair_probs, data["Outcomes"])
print(f"Brier Score: {brier_score:.4f}")

# Get profit from betting on favourites
profits, total_wins = simulate_favourite_betting_strategy(fair_odds, fair_probs, data["Outcomes"])
print(f"Total Profit from Betting on Favourites: {sum(profits):.2f}")
print(f"Total Wins: {total_wins} out of {len(data)} matches")

Brier Score: 0.3873
Total Profit from Betting on Favourites: 169.40
Total Wins: 2280 out of 3264 matches


In [ ]:
# Get Weighted Odds
weighted_odds, weighted_probs = calculate_weighted_implied_probabilities(data[['Home Odds', 'Away Odds']])